In [1]:
# No es necesario Tensorflow
!pip uninstall -y tensorflow
# Instalar transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
!pip install transformers[torch] accelerate -U

Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
tokenizers                               0.19.1
transformers                             4.41.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1


In [2]:
import os
!pip install gdown

In [3]:
#Descargar archivos de google Drive
import gdown
# Lista de archivos necesarios
files = [
    ("16SkLOsfja22kIwExs4NiU5pjrOV7SUdP", "pytorch_model.bin"),
    ("1PrM9LMJ9Pmrc8yqKBT1OMRPXD1urkJ1r", "merges.txt"),
    ("1i6L13u5P9HVzzmKsNZxe_wICteulIWY5", "vocab.json"),
    ("1lDaVeJc90TKbBrhxZKZbIfRTPv9VSsOg", "config.json")
]
folder_name = "quechuaBERT"
os.makedirs(folder_name, exist_ok=True)

# Descargar
for file_id, file_name in files:
    destination_path = os.path.join(folder_name, file_name)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", destination_path, quiet=False)
#tokenizer.save_model("quechuaBERT")

Downloading...
From (original): https://drive.google.com/uc?id=16SkLOsfja22kIwExs4NiU5pjrOV7SUdP
From (redirected): https://drive.google.com/uc?id=16SkLOsfja22kIwExs4NiU5pjrOV7SUdP&confirm=t&uuid=c15f1302-9658-4dc6-a0b6-1ad04aca42de
To: /kaggle/working/quechuaBERT/pytorch_model.bin
100%|██████████| 334M/334M [00:01<00:00, 219MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1PrM9LMJ9Pmrc8yqKBT1OMRPXD1urkJ1r
To: /kaggle/working/quechuaBERT/merges.txt
100%|██████████| 483k/483k [00:00<00:00, 113MB/s]
Downloading...
From: https://drive.google.com/uc?id=1i6L13u5P9HVzzmKsNZxe_wICteulIWY5
To: /kaggle/working/quechuaBERT/vocab.json
100%|██████████| 837k/837k [00:00<00:00, 51.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lDaVeJc90TKbBrhxZKZbIfRTPv9VSsOg
To: /kaggle/working/quechuaBERT/config.json
100%|██████████| 676/676 [00:00<00:00, 2.69MB/s]


In [4]:
#Descargar el corpus de nuestro repositorio https://github.com/Xnehil/TACC-Lexemas/blob/main/data/corpus/corpus.csv
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Xnehil/TACC-Lexemas/main/data/corpus/corpus.csv")
df.head()

#Y guardar un txt; una fila por 'sentence' del dataframe
with open('corpus.txt', 'w') as f:
    for sentence in df['sentence']:
        f.write(sentence + '\n')

In [5]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

## QuBERT

Toda esta sección está basada en el código del repositorio https://github.com/Llamacha/QuBERT. Este modelo fue presentado en el paper [Introducing QuBERT: A Large Monolingual Corpus and BERT Model for Southern Quechua](https://aclanthology.org/2022.deeplo-1.1) (Zevallos et al., DeepLo 2022)

In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# Cargar el tokenizador de quechuaBERT
tokenizer = ByteLevelBPETokenizer(
    "./quechuaBERT/vocab.json",
    "./quechuaBERT/merges.txt",
)

In [7]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [8]:
tokenizer.encode("allinllachu manan allinlla huk wasipita").tokens

['<s>',
 'allin',
 'llachu',
 'Ġmanan',
 'Ġallinlla',
 'Ġhuk',
 'Ġwasipi',
 'ta',
 '</s>']

In [9]:
# Verificar GPU
!nvidia-smi

Fri Jun 21 19:39:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [10]:
# Verificar que PyTorch lo vea
import torch
torch.cuda.is_available()

True

In [11]:
from transformers import RobertaConfig

# configuración de Roberta 
config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [12]:
import json
tokenizer_config = {"max_len": 512}

with open("./quechuaBERT/tokenizer_config.json", 'w') as fp:
    json.dump(tokenizer_config, fp)

In [13]:
from transformers import RobertaTokenizerFast

# Cargar el tokenizador para el modelo
tokenizer = RobertaTokenizerFast.from_pretrained("./quechuaBERT", max_len=512)

In [14]:
from transformers import RobertaForMaskedLM, RobertaForCausalLM, RobertaTokenizer

# El modelo original
model_mlm = RobertaForMaskedLM.from_pretrained('./quechuaBERT')

# Las siguientes modificaciones fueron hechas para la generación de textos
# RobertaFoCausal es para generación
model_clm = RobertaForCausalLM.from_pretrained('./quechuaBERT')

# Copiar los pesos del model MLM al modelo CLM
model_clm.roberta = model_mlm.roberta

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


In [15]:
model_clm.num_parameters()

123440416

In [16]:
%%time
from transformers import LineByLineTextDataset

# Cargar corpus generado a Dataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./corpus.txt",
    block_size=128,
)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 4.04 s, sys: 386 ms, total: 4.43 s
Wall time: 2.09 s


In [17]:
from transformers import DataCollatorForLanguageModeling

# Generar el DataCollator con el tokenizador
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

In [19]:
from transformers import Trainer, TrainingArguments
import os

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none",
)

# Declarar trainer con parámetros necesarios
trainer = Trainer(
    model=model_clm,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [20]:
# para evitar errores con cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# entrenar modelo
trainer.train()

In [21]:
# guardar modelo en directorio prueba
trainer.save_model("./prueba")

In [39]:
# Codificar el prompt de entrada
input_ids = tokenizer.encode("", return_tensors='pt').to(device)

# Generar texto
output = model_clm.generate(input_ids, max_length=40)

# Decodificar texto generado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


 miimimi


## GPT -2

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

# Para evitar errores con cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar el tokenizador de gpt2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Cargar el modelo 
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [7]:
# Función de carga del corpus
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

file_path = './corpus.txt'

# Cargar y tokenizar el corpus en el dataset
dataset = load_dataset(file_path, tokenizer)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
# definir DataCollator con el tokenizador
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_dir='./logs',
    logging_steps=200,
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
# Entrenar el modelo
trainer.train()

# Guardar el modelo y el tokenizador
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

In [10]:
# prompt de entrada
input_prompt = "Yaqapis"

# Codificar prompt de entrada
input_ids = tokenizer.encode(input_prompt, return_tensors='pt').to(device)
attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)

# generar texto
output = model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2, top_p=0.9, temperature=0.8,
                        pad_token_id=tokenizer.eos_token_id)

# Dedodificar el texto generado
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Texto generado:")
print(generated_text)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Texto generado:
Yaqapisqanpaq, ichaqa kasqanku, chaytaq   chakunapi, kutichiykipi, hinaspa  kachkanki.
Chay  huk llaqtapasqa, pacha  qipaqta, qillqas  yachakusqanta, yuyaykamanta  taytataq.


## gpt2-genre-generator

Este modelo fue sacado de https://huggingface.co/aspis/gpt2-genre-story-generation. Parte del código fue basado en sus instrucciones de uso.

In [25]:
from transformers import pipeline, TextGenerationPipeline, GPT2LMHeadModel, AutoTokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
import torch

# para evitar errores con el cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cargar modelo y tokenizador
model_source = "aspis/gpt2-genre-story-generation"
model = GPT2LMHeadModel.from_pretrained(model_source)
tokenizer = AutoTokenizer.from_pretrained(model_source)

In [26]:
# Función para cargar el corpus
def load_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

file_path = './corpus.txt'

# Cargar y tokenizar el dataset
dataset = load_dataset(file_path, tokenizer)

/opt/conda/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [30]:
# Declarar DataCollator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Inicializar el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
# Entrenar el modelo
trainer.train()

# Guardar modelo y tokenizador
model.save_pretrained('./story_generator_fined_tuned')
tokenizer.save_pretrained('./story_generator_fined_tuned')

In [22]:
# Declarar el generador con pipeline de transformers
generator = TextGenerationPipeline(model=model, tokenizer=tokenizer)

In [26]:
# prompt de entrada 
input_prompt = "<BOS> <adventure> Allinllachu" # <adventure> indica el género de la entrada
story = generator(input_prompt, max_length=150, do_sample=True,
               repetition_penalty=1.5, temperature=1.2, 
               top_p=0.95, top_k=50)
print(story)

[{'generated_text': '<BOS> <adventure> Allinllachu, katahina wakinaykunaq hampiwas pisachata quraqtichakuqa rimapunchim?\nChirikarispa quywanakumupi willapa rurasinchuta munipita chaytata runaspan allintam uyyuri manman iskayankuñullun. llananña tapukuykus sukhusquymantinkaqlla maywaspa wasitunkuman ayllurqapi yana aykunuspallopiyka ñuit ninsha urkarim mittanchikpima urvinanchu, llamananta sawarchiyta pu'}]
